In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
ticker = "SPY"

r = pd.read_csv("../data/interim/log_returns.csv", index_col=0, parse_dates=True)

r.head()

/tmp/ipykernel_3592/3126732154.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  r = pd.read_csv("../data/interim/log_returns.csv", index_col=0, parse_dates=True)


,log_return
1,-0.000164
2,-0.002902
3,0.006123
4,0.000218
5,0.000654


In [4]:
r.shape

(3042, 1)

# Backtesting Settings

In [9]:
window = 1000
alpha = 0.99

# Define VaR Functions

### Normal VaR

In [5]:
def normal_var(data, alpha):
    mu = data.mean()
    sigma = data.std()
    return stats.norm.ppf(1 - alpha, mu, sigma)

### Student-t VaR

In [6]:
def t_var(data, alpha):
    nu, mu, sigma = stats.t.fit(data)
    return stats.t.ppf(1 - alpha, nu, mu, sigma)

### Skew VaR

In [7]:
def skew_var(data, alpha):
    a, mu, sigma = stats.skewnorm.fit(data)
    return stats.skewnorm.ppf(1 - alpha, a, mu, sigma)

# Rolling VaR Forecast

In [10]:
var_normal = []
var_t = []
var_skew = []
actual_returns = []

for i in range(window, len(r)):
    sample = r.iloc[i-window:i]

    var_normal.append(normal_var(sample, alpha))
    var_t.append(t_var(sample, alpha))
    var_skew.append(skew_var(sample, alpha))
    actual_returns.append(r.iloc[i])

var_normal = np.array(var_normal)
var_t = np.array(var_t)
var_skew = np.array(var_skew)
actual_returns = np.array(actual_returns)

# Count Breaches

In [11]:
breach_normal = actual_returns < var_normal
breach_t = actual_returns < var_t
breach_skew = actual_returns < var_skew

print("Expected breach rate:", 1 - alpha)
print("Normal breach rate:", breach_normal.mean())
print("Student-t breach rate:", breach_t.mean())
print("Skew breach rate:", breach_skew.mean())

Expected breach rate: 0.010000000000000009
Normal breach rate: 0.02693437806072478
Student-t breach rate: 0.015300626126562559
Skew breach rate: 0.02335455915490661
